In [1]:
from datetime import timedelta
import torch
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader,random_split
from __future__ import print_function
import numpy as np 
import pandas as pd
import os
import time
import random
import matplotlib.pyplot as plt 
from torch.autograd import Function
from collections import OrderedDict
import torch.nn as nn
import math
import torchvision.models as models
import pickle
import cv2
import os
import sys
import glob
import collections
import pandas as pd
import wandb
from sklearn.model_selection import KFold
# from torchsummary import summary
import glob
from sklearn.manifold import TSNE
import seaborn as sns
import collections
from transformers import BertConfig, BertModel

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import tokenizers
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, LineByLineTextDataset, BertModel, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

SAVING_FRAMES_PER_SECOND = 30

In [2]:
VIDEO_FOLDER = "/cifs/data/tserre/CLPS_Serre_Lab/nih/files_to_send/"
ACTION_FOLDER = "/cifs/data/tserre/CLPS_Serre_Lab/nih/inference/results_csv/"
CONDENSED_ACTION_FOLDER = "/cifs/data/tserre/CLPS_Serre_Lab/anagara8/prj_nih/Notebooks/CondensedActions/"
condensed_action_folders = os.listdir(CONDENSED_ACTION_FOLDER)

In [3]:
BEH_LABELS = ['drink', 'eat', 'groom', 'hang', 'sniff', 'rear', 'rest', 'walk', 'nibble']
action_colours = ['tab:blue', 'tab:orange', 'limegreen', 'deepskyblue', 'tab:pink', 'tab:purple', 'tab:brown', 'red', 'black']
len(action_colours) == len(BEH_LABELS)

True

In [4]:
def cohort_search(cohort, search_root, ret=False):
    cohort = str(cohort).upper()
    all_folders = [folder for folder in os.listdir(search_root) if folder.startswith("FC-"+cohort)]
    all_folders.sort()
    all_folder_list = "\n".join(all_folders)
    if ret:
        return all_folders
    else:
        print(all_folder_list)

In [ ]:
def find_mouse_videos(mouse_id, cohort, ret=False):
    
    search_for = "action"
    
    if search_for == "videos":
        print("Looking for videos..")
        file_extension = ".mp4" 
        ROOT_FOLDER = VIDEO_FOLDER
    else:
        print("Looking for actions...")
        file_extension = ".csv"
        ROOT_FOLDER = ACTION_FOLDER
    
    mouse_id = str(mouse_id)
    cohort_folders = cohort_search(cohort, ROOT_FOLDER, ret=True)
    
    preexposure_folders = [folder for folder in cohort_folders if "exp" in folder]
    postcond_folders = [folder for folder in cohort_folders if "cond" in folder]
    postext_folders = [folder for folder in cohort_folders if "xt" in folder]
    postret_folders = [folder for folder in cohort_folders if "tre" in folder]
    
    preexposure_videos = []
    postcond_videos = []
    postext_videos = []
    postret_videos = []
    
    for folder in preexposure_folders:
        preexposure_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    preexposure_videos.sort()
    
    for folder in postcond_folders:
        postcond_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postcond_videos.sort()
    
    for folder in postext_folders:
        postext_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postext_videos.sort()
    
    for folder in postret_folders:
        postret_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postret_videos.sort()
    
    if ret:
        return preexposure_videos, postcond_videos
    else:
        print("\nPREEXPOSURE:")
        print("\n".join(preexposure_videos))
        print("\nPOSTCONDITIONING:")
        print("\n".join(postcond_videos))
        print("\nPOSTEXTINCTION:")
        print("\n".join(postext_videos))
        print("\nPOSTRETRIEVAL:")
        print("\n".join(postret_videos))

In [ ]:
# time_counter = {
#     'under5':0,
#     '5-10':0,
#     '10-15':0,
#     '15-20':0,
#     '20-25':0,
#     '25-30':0,
#     '30-35':0,
#     '35-40':0,
#     '40-45':0,
#     '45-50':0,
#     '50-55':0,
#     '55-60':0,
#     '60-90':0,
#     '90-450':0,
#     '450-1800':0,
#     '1800-4500':0,
#     'over4500':0,}

In [ ]:
# def get_repeat_vocab(repeat_number):
#     # > '0-5'
#     # > '5-15'
#     # > '15-45'
#     # > '45-90'
#     # > '90-450'
#     # > '450-1800'
#     # > '1800-4500'
#     # > 'over4500'
#     if repeat_number > 0 and repeat_number <= 5:
#         box = "under5"
#     elif repeat_number > 5 and repeat_number <= 10:
#         box = "5-10"
#     elif repeat_number > 10 and repeat_number <= 15:
#         box = "10-15"
#     elif repeat_number > 15 and repeat_number <= 20:
#         box = "15-20"
#     elif repeat_number > 20 and repeat_number <= 25:
#         box = "20-25"
#     elif repeat_number > 25 and repeat_number <= 30:
#         box = "25-30"
#     elif repeat_number > 30 and repeat_number <= 35:
#         box = "30-35"
#     elif repeat_number > 35 and repeat_number <= 40:
#         box = "35-40"
#     elif repeat_number > 40 and repeat_number <= 45:
#         box = "40-45"
#     elif repeat_number > 45 and repeat_number <= 50:
#         box = "45-50"
#     elif repeat_number > 50 and repeat_number <= 55:
#         box = "50-55"
#     elif repeat_number > 55 and repeat_number <= 60:
#         box = "55-60"
#     elif repeat_number > 60 and repeat_number <= 90:
#         box = "60-90"
#     elif repeat_number > 90 and repeat_number <= 450:
#         box = "90-450"
#     elif repeat_number > 450 and repeat_number <= 1800:
#         box = "450-1800"
#     elif repeat_number > 1800 and repeat_number <= 4500:
#         box = "1800-4500"
#     elif repeat_number > 4500:
#         box = "over4500"
#     else:
#         raise ValueError('A very specific bad thing happened.')
#     time_counter[box] += 1
#     return box

In [ ]:
# def cleaning_actions(action_list):
#     WINDOW_SIZE = 3 # Each side
    
#     index = WINDOW_SIZE
#     action_length = len(action_list)
    
#     while index < (action_length - WINDOW_SIZE - 1):
#         current_action = action_list[index]
#         window = action_list[index - WINDOW_SIZE : index + WINDOW_SIZE + 1]
#         window_elements = collections.Counter(window)
#         most_common_action = window_elements.most_common(1)[0][0]
#         if action_list[index] != most_common_action:
#             action_list[index] = most_common_action
#         index += 1
#     return action_list

In [ ]:
# def generate_language_from_labels(action_list, class_label):
#     SENTENCE_LENGTH = 900 # in terms of frames
    
#     start_index = 0
#     action_length = len(action_list)
    
#     language = []
# #     label = []
#     sentence = ""
    
#     current_sentence_length = 0
#     while start_index < action_length:
        
#         current_action = action_list[start_index]
#         current_action_label = BEH_LABELS[current_action]
        
#         next_action_idx = start_index + 1
        
#         while next_action_idx < action_length and current_action == action_list[next_action_idx]:
#             next_action_idx  += 1
#         repeats = next_action_idx - start_index
        
#         if current_sentence_length == 0:
#             sentence = ""
            
#         current_sentence_length += repeats
#         start_index = next_action_idx
        
#         sentence += str(current_action_label)+" "+get_repeat_vocab(repeats)+" "
        
#         if current_sentence_length >= SENTENCE_LENGTH:
#             language.append(sentence)
# #             label.append(class_label)
#             current_sentence_length = 0
            
#     if sentence != "":
#         language.append(sentence)
# #         label.append(class_label)
#     return language

In [ ]:
# def get_label(filename):
#     if "exp" in filename:
#         return 0
    
#     if "cond" in filename:
#         return 1
    
#     if "xt" in filename:
#         return 2
    
#     if "tr" in filename:
#         return 3
#     else:
#         raise ValueError

In [ ]:
# Storing as Individual Action Files

# folder_counter = 1
# for folder in folder_list:
#     print("\nFOLDER:", str(folder_counter)+"/"+str(len(folder_list)))
#     folder_counter += 1
#     action_files = os.listdir(ACTION_FOLDER + folder)
#     csv_counter = 1
#     for csv_name in action_files:
#         actions = pd.read_csv(ACTION_FOLDER + folder + '/' + csv_name)
#         print("\n"+str(csv_counter)+"/"+str(len(action_files))+" Reading CSV:", csv_name)
#         csv_counter += 1
#         current_actions = []
#         for idx in range(len(actions)):
#             current_actions.append(actions.iloc[idx][1])
#         start = time.time()
#         cleaned_actions = cleaning_actions(current_actions)
#         cleaning = time.time()
#         condensed_actions = generate_language_from_labels(cleaned_actions, get_label(folder))
#         condensing = time.time()
#         save_name = csv_name.split('.csv')[0] + ".npy"
#         saving = time.time()
#         try:
#             np.save("CondensedActions/"+folder+"/"+save_name, condensed_actions)
#         except:
#             os.mkdir("CondensedActions/"+folder)
#             np.save("CondensedActions/"+folder+"/"+save_name, condensed_actions)
#         print("Actions for "+folder+"/"+csv_name+" processed. Label:", get_label(folder))
#         print("Times:: Cleaning:", cleaning-start, " Condensing:", condensing-cleaning, "Saving:", saving-condensing, " Total:", saving-start)

In [ ]:
# Storing as Folders

# folder_counter = 1

# for folder in condensed_action_folders:
#     all_sequences = np.array([])
#     all_labels = np.array([])
#     print("\nFOLDER:", str(folder_counter)+"/"+str(len(condensed_action_folders)))
#     folder_counter += 1
#     action_files = os.listdir(CONDENSED_ACTION_FOLDER + folder)
#     csv_counter = 1
#     start = time.time()
#     print(folder)
#     for csv_name in action_files:
#         actions = np.load(CONDENSED_ACTION_FOLDER + folder + '/' + csv_name)
#         csv_counter += 1
#         all_sequences = np.append(all_sequences, actions)
#         all_labels = np.append(all_labels, [get_label(folder)]*len(actions))
#         if len(all_sequences) != len(all_labels):
#             raise ValueError
#     print("Append Time:", time.time()-start)
#     np.save("./FolderWise/Sequences_"+folder, all_sequences)
#     np.save("./FolderWise/Labels_"+folder, all_labels)

In [ ]:
# Reading folders and appending:

all_sequences = np.array([])
all_labels = np.array([])

folder_counter = 0
for folder in condensed_action_folders:
    start = time.time()
    folder_counter += 1
    
    sequences = np.load("./FolderWise/Sequences_"+folder+".npy", allow_pickle=True)
    labels = np.load("./FolderWise/Labels_"+folder+".npy", allow_pickle=True)
    
    all_sequences = np.append(all_sequences, sequences)
    all_labels = np.append(all_labels, labels)
    
    csv_counter = 1
    print("\nFOLDER:", )
    print(str(folder_counter)+"/"+str(len(condensed_action_folders)),"Append Time:", time.time()-start)
    

In [ ]:
len(all_sequences), len(all_sequences) == len(all_labels)

In [ ]:
max_length = -1
seq_lens = []
for sequence in all_sequences:
    l = len(sequence.split(' '))
    seq_lens.append(l)
    if l > max_length:
        max_length = l
print("Maximum sequence length:", max_length)
seq_lens = np.array(seq_lens)
print("Average:", np.mean(seq_lens))
print("Standard Deviation:", np.std(seq_lens))

# To ensure we don't miss anything, we will use 512

# BERT

In [1]:
!cat nih_vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
under5
5-10
10-15
15-20
20-25
25-30
30-35
35-40
40-45
45-50
50-55
55-60
60-90
90-450
450-1800
1800-4500
over4500
drink
eat
groom
hang
sniff
rear
rest
walk
nibble

In [ ]:
from tqdm import tqdm
from transformers import BertTokenizer

# create a python generator to dynamically load the data
def batch_iterator(batch_size=100):
    for i in tqdm(range(0, len(raw_text), batch_size)):
        print("Batch:", raw_text[i : i + batch_size])
        yield raw_datasets[i : i + batch_size]

tokenizer = BertTokenizer("nih_vocab.txt", do_basic_tokenize=False)

In [ ]:
example_text = 'sniff 20-25 groom under5 sniff 5-10 groom 10-15 sniff 40-45 groom 20-25 sniff 5-10 groom 20-25 sniff under5 groom 50-55 sniff 90-450 groom 20-25 sniff 60-90 groom 90-450 sniff 15-20 groom 30-35 sniff 55-60 groom 60-90 sniff 60-90 nibble 10-15 sniff 15-20 nibble 45-50 sniff 50-55'
bert_input = tokenizer(example_text,padding='max_length', max_length = 512, 
                       truncation=True, return_tensors="pt")

In [ ]:
def print_rand_sentence():
    '''
    Displays the tokens and respective IDs of a random text sample
    '''
    index = random.randint(0, len(all_sequences)-1)
    table = np.array([tokenizer.tokenize(all_sequences[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(all_sequences[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

In [ ]:
# Convert data into required format
dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = 'all_30s_sequences.txt',
    block_size = 256
)
print('No. of lines: ', len(dataset))

In [8]:
# Defining configuration of BERT for pretraining
config = BertConfig(
    vocab_size=31,
    hidden_size=768, 
    num_hidden_layers=6, 
    num_attention_heads=12,
    max_position_embeddings=256
)

In [ ]:
model = BertForMaskedLM(config)
print('No of parameters: ', model.num_parameters())


In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
# Defining training configuration\
training_args = TrainingArguments(
    output_dir='.',
    overwrite_output_dir=True,
    num_train_epochs=7,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Perfrom pre-training and save the model
trainer.train()
trainer.save_model('.')

In [9]:
a = BertForSequenceClassification(config)

In [10]:
a

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31, 768, padding_idx=0)
      (position_embeddings): Embedding(256, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis